# Network Traffic Analysis Pipeline

## 0. Setup Environment

In [ ]:
%pip install pyshark
%pip install nest_asyncio
%pip install pandas


Allow the notebook to work assynchronously.

In [ ]:
import nest_asyncio
nest_asyncio.apply()

## 1. Data Gathering

In [ ]:
dataset_files = ['data/2024-10-23-Redline-Stealer-infection-traffic.pcap']

## 2. Data Preprocessing

1. Import the desired preprocessor.

In [ ]:
from src.preprocessors.pcap_preprocessor import PcapPreprocessor

preprocessor = PcapPreprocessor()

2. Load the datasets into a dataframe.

In [ ]:
base_data = preprocessor.load_datasets(dataset_files)
print(base_data)

3. Preprocess the dataframe.

In [ ]:
# print all values of the first 5 rows as text
print(base_data.head(5).to_string())
#processed_data = preprocessor.preprocess_dataframe(base_data)

4. Partition the dataframe into training and testing sets.

In [ ]:
base_training_percentage = 0.6
online_training_percentage = 0.2
validation_percentage = 0.2

training_data, online_training_data, validation_data = preprocessor.split_dataframe(processed_data, base_training_percentage, online_training_percentage, validation_percentage)

### Visualization

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from src.visualization.visualization import Visualization

visualization = Visualization()

# Check available columns in base_data
print("Columns in base_data:", base_data.columns.tolist())

In [ ]:
visualization.plot_missing_values_heatmap(base_data)

In [ ]:
visualization.plot_tcp_source_ports(base_data)

In [ ]:
visualization.plot_udp_source_ports(base_data)

In [ ]:
visualization.plot_top_ip_addresses(base_data)

In [ ]:
visualization.plot_top_ip_dest_addresses(base_data)

In [ ]:
visualization.plot_tcp_flags_distribution(base_data)

In [ ]:
visualization.plot_dns_queries(base_data)

In [ ]:
visualization.plot_correlation_matrix(base_data)

In [ ]:
visualization.plot_tcp_stream_time_series(base_data)

## 3. Model Training

1. Select the desired models.

In [ ]:
from src.models.offline import OfflineModel
from src.models.online import OnlineModel

model_list = [OfflineModel(), OnlineModel()]

2. Train the models according to their type.

In [ ]:
for model in model_list:
    model.train(training_data)
    if model.is_online():
        model.predict_batch(online_training_data)

## 4. Model Evaluation

1. Get the predictions from the models into a dataframe.

In [ ]:
# Header
header = ['expected']
for model in model_list:
    header.append(model.get_name())
validation_results = [header]

# Results
for index, row in validation_data.iterrows():
    line = [row['label']]
    for model in model_list:
        line.append(model.predict(row))
    validation_results.append(line)
 
df = pd.DataFrame(validation_results)

2. Select the desired evaluator.

In [ ]:
from src.evaluators.standard_evaluator import Evaluator

evaluator = Evaluator()

3. Evaluate the models using the predictions.

In [ ]:
results = evaluator.evaluate(df)

4. Analyze the results.

In [ ]:
print(results)